# Архив новостей по акциям MOEX

In [1]:
# Библиотеки
import pandas as pd
import numpy as np

import requests as rq
import urllib.request
from urllib.parse import urlencode
from urllib.request import urlopen

from bs4 import BeautifulSoup as bs
import json
import re
import io
import os
import time

import datetime as dt

import datetime
from datetime import datetime, date, timedelta

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

## Итерфакс

In [2]:
def date_range(start, end, intv):
    start = datetime.strptime(start,"%Y%m%d")
    end = datetime.strptime(end,"%Y%m%d")
    diff = (end  - start ) / intv
    for i in range(intv):
        yield (start + diff * i).strftime("%Y/%m/%d")
    yield end.strftime("%Y/%m/%d")

In [9]:
def parse_interfax_news(data_end = '2022/02/01', data_start ='2022/01/01' ):
   
    ress = []
    
#     if days == 0:
#         data_list = [str(date.today()).replace("-", "/")]
#     else:
    data_end = str(data_end).replace("/", "")
    data_start = str(data_start).replace("/", "")
    data_list= list(date_range(data_start, data_end, 1))
    
    CLEANR = re.compile('<.*?>') 
    
    
    for url in data_list:
        interfax_business_url = 'https://www.interfax.ru/news/' + url
        n = rq.get(interfax_business_url).content
        soup = bs(n, 'html.parser')
        n = soup.body.find('div', {'class':'pages'})
        news_urls = re.findall('/news/'+ url+'/all/\w+', str(n))
        news_urls = list(map(lambda x: 'https://www.interfax.ru' + x, news_urls))
        news_urls.insert(0,interfax_business_url ) 
        
        
        for ur in news_urls:
                    response = rq.get(ur)
                    news_u = re.findall('/business/\w+', response.text)
                    news_u = list(map(lambda x: 'https://www.interfax.ru' + x, news_u))
                    
                    
                    for u in news_u:
                                res = {}
                                nv = rq.get(u).content
                                s = bs(nv, 'html.parser')
                                res['time'] = s.find('meta', {'itemprop': 'datePublished'})
                                res['time'] = str(re.findall('\d{4}-\d+-\d+T\d+:\d+:\d+', str(res['time']))[0]).replace("T", " ")
                                res['url']= u
                                res['site'] = 'interfax_business'
                                res['title'] = s.body.find(itemprop="headline").text
                                res['text'] = s.body.find('article', {"itemprop":"articleBody"}).find_all('p')
                                res['text'][0] = ' '.join(str(res['text'][0]).split(' ')[5:])
                                res['text'] = re.sub(CLEANR, '', str(res['text'])).replace("\'", "").replace("]", "").replace("[", "").replace(".,", ".")
                                ress.append(res)   
     
    
    ress = [i for j, i in enumerate(ress) if i['time'][:10].replace("-", "/") in data_list]

    
    return ress

In [23]:
m1 = ['2022/01/01', '2022/02/01', '2022/03/01', '2022/04/01', '2022/05/01', '2022/06/01', '2022/07/01','2022/08/01', '2022/09/01', '2022/10/01', '2022/11/01', '2022/12/01']
m2 = ['2023/01/01','2023/02/01', '2023/03/01', '2023/04/01', '2023/05/01', '2023/06/01', '2023/07/01','2023/08/01', '2023/09/01', '2023/10/01','2023/10/23']
m = m1 + m2
y = list(zip(m[:-1], m[1:]))
df = pd.DataFrame()
for i, j in tqdm(y):
        data_end = i
        data_start = j
        res = parse_interfax_news(data_end, data_start)
        dfin = pd.json_normalize(res)
        df = df.append(dfin)
        time.sleep(20)

  0%|          | 0/22 [00:00<?, ?it/s]

In [28]:
df

,time,url,site,title,text
0,2022-02-01 22:01:00,https://www.interfax.ru/business/819685,interfax_business,"Газопровод ""Ямал-Европа"" в полночь должен верн...","Газопровод ""Ямал-Европа"" в полночь со вторника..."
1,2022-02-01 20:49:00,https://www.interfax.ru/business/819674,interfax_business,Техкомитет ОПЕК+ спрогнозировал низкие коммерч...,Технический комитет ОПЕК+ рассмотрел три возмо...
2,2022-02-01 19:59:00,https://www.interfax.ru/business/819664,interfax_business,Россия со 2 февраля на два месяца запретит экс...,Россия со 2 февраля вводит двухмесячный запрет...
3,2022-02-01 19:45:00,https://www.interfax.ru/business/819659,interfax_business,Рубль вечером продолжил укрепляться к доллару ...,"Доллар и евро продолжают заметно дешеветь на ""..."
4,2022-02-01 19:33:00,https://www.interfax.ru/business/819657,interfax_business,Рынок акций РФ вечером растерял большую часть ...,Рынок акций РФ во вторник вырос на фоне локаль...
...,...,...,...,...,...
51,2023-10-23 08:01:00,https://www.interfax.ru/business/927099,interfax_business,"Сменился владелец российской ""дочки"" зернотрей...","Владельцем российской ""дочки"" зернотрейдера Vi..."
52,2023-10-23 06:26:00,https://www.interfax.ru/business/927094,interfax_business,Дефицит бюджета США в 2023 финансовом году уве...,Дефицит госбюджета США по итогам 2023 финансов...
53,2023-10-01 15:14:00,https://www.interfax.ru/business/923623,interfax_business,Будущий запрет на экспорт топлива из РФ будет ...,Будущий запрет на экспорт топлива из России бу...
54,2023-10-01 14:45:00,https://www.interfax.ru/business/923620,interfax_business,Власти РФ проследят за снижением цен на заправках,Правительство РФ намерено проследить за снижен...


In [29]:
df.to_csv('interfax_business_new.csv', index = False)

## РБК

In [39]:
RBC_FINANCE_URL = 'https://rbc.ru/v10/ajax/get-news-by-filters/?category=finances&limit=20&offset='
CLEANR = re.compile('<.*?>') 

In [41]:
t = time.sleep(10)

def parse_rbc_news(days=1):
    offset = 0
    ress = []
    while (True):
        global t
        response = rq.get(RBC_FINANCE_URL + str(offset))
        offset += 20
        if (response.status_code == 200):
            news_urls = re.findall('https://www.rbc.ru/finances/\d+/\d+/\d+/\w+', response.text)
            for url in news_urls:
                try:
                    res = {}
                    res['url']= url
                    news = rq.get(url).text
                    soup = bs(news, 'html.parser')
                    res['site'] = 'rbc_finances'
                    res['title'] = soup.body.find(class_="article__header__title-in js-slide-title").text.replace("\xa0", " ").replace("\t", " ").strip()
                    
                    def cleanhtml(raw_html):
                        return re.sub(CLEANR, '', raw_html)

                    raw = soup.body.find('div', class_="article__text article__text_free").find_all('p')
                    res['text'] = ''
                    for i in raw:
                        if (str(i).find('span') == -1):
                            res['text'] += cleanhtml(str(i))
                    try:
                        res['description'] = soup.body.find(class_="article__text__overview").text.replace("\xa0", " ").replace("\t", " ")
                    except:
                        if (len(res['text']) > 100):
                            res['description'] = res['text'][:100] + '...'
                        else:
                            res['description'] = res['text']
                    date_raw =re.findall('\d{4}-\d+-\d+T\d+:\d+:\d+' ,str(soup.body.find(class_="article__header__date")))[0]
                    date, time = date_raw.split('T')
                    date = date.split('-')
                    time = time.split(':')
                    res['timestamp'] = dt.datetime(int(date[0]), int(date[1]), int(date[2]), int(time[0]), int(time[1]), int(time[2])).timestamp()
                    res['time'] = str(date_raw).replace("T", " ")
                    if (days * 86400 + res['timestamp'] < dt.datetime.now().timestamp()):
                        return ress
                    ress.append(res)
                except Exception as e:
                    print(e, url)
        else:
            return ress

In [47]:
%time
res1 = parse_rbc_news(730)

CPU times: total: 0 ns
Wall time: 0 ns
("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)) https://www.rbc.ru/finances/16/08/2023/64dcbb469a7947ebf39bf721
("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)) https://www.rbc.ru/finances/15/08/2023/64db4bea9a79476a37062dfb
("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)) https://www.rbc.ru/finances/14/08/2023/64d9c9889a7947f4fa85ad90
("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)) https://www.rbc.ru/finances/11/08/2023/64d630809a7947ffaef1441e
("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)) https://www.rbc.ru/finances/07/06/2023/647f6ae49a7947ab74c73a04


In [48]:
df1 = pd.json_normalize(res1)
df1 = df1[['time','url', 'site', 'title', 'text']]
df1['text'] = df1['text'].replace(r'\s+|\\n', ' ', regex=True)
df1

,time,url,site,title,text
0,2023-10-23 13:56:38,https://www.rbc.ru/finances/23/10/2023/6536315...,rbc_finances,Российские банки заняли более 10% мирового рын...,Доля крупнейших российских банков-эквайеров в ...
1,2023-10-23 00:01:07,https://www.rbc.ru/finances/23/10/2023/6532746...,rbc_finances,"Эксперты оценили, на сколько ЦБ повысит ключев...",Совет директоров Банка России на заседании 27 ...
2,2023-10-23 00:00:55,https://www.rbc.ru/finances/23/10/2023/65317f7...,rbc_finances,Банки не поддержали законопроект о доступе спе...,"Российские банки не поддержали законопроект, к..."
3,2023-10-22 08:00:28,https://www.rbc.ru/finances/22/10/2023/65329cd...,rbc_finances,Как число россиян-банкротов превысило 1 млн. И...,Число россиян-банкротов в России превысило 1 м...
4,2023-10-21 07:00:08,https://www.rbc.ru/finances/21/10/2023/6532939...,rbc_finances,"Россияне впервые за 1,5 года снизили объем нал...",Россияне впервые с начала года сократили объем...
...,...,...,...,...,...
803,2022-12-22 18:11:32,https://www.rbc.ru/finances/22/12/2022/63a471f...,rbc_finances,Приложение Россельхозбанка удалили из AppStore...,Apple удалила приложение Россельхозбанка из ма...
804,2022-12-22 14:34:16,https://www.rbc.ru/finances/22/12/2022/63a434e...,rbc_finances,Новым главой «Открытия» станет бывший заместит...,Михаил Задорнов покинет должность председателя...
805,2022-12-22 12:42:20,https://www.rbc.ru/finances/22/12/2022/63a4267...,rbc_finances,ЦБ объявил о продаже «Открытия» банку ВТБ за ₽...,Совет директоров Центробанка официально принял...
806,2022-12-22 10:59:04,https://www.rbc.ru/finances/22/12/2022/63a3ffd...,rbc_finances,Приложения Росбанка пропали из App Store после...,Приложения Росбанка после введения санкций про...


In [45]:
df1.to_csv('rbc_finances_new.csv', index = False)

## Smartlab

In [2]:
def get_date(date):
    month_list = {'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04', 'мая': '05', 'июня': '06',
           'июля': '07', 'августа': '08', 'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'}
    date_list = date.split(' ')
    month = month_list[date_list[1]]
    return date_list[2] +'-' + month + '-'+ date_list[0] + ' ' + date_list[-1] + ':00'

In [3]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [4]:
t1 = time.sleep(20)
t2 = time.sleep(10)

In [67]:
def parse_smartlab_news():
    
        ress = []
        n = rq.get('https://smart-lab.ru/forum/news/SBER')
        news_n = re.findall('/forum/news/SBER/page\w+', n.text)
        news_n = int(re.findall('[0-9.]+', news_n[-1])[0])

        for url in range(1, news_n):
                    try:   
                        global t1 

                        business_url = 'https://smart-lab.ru/forum/news/SBER/page' + str(url)
                        rn = rq.get(business_url)
                        news_urls = re.findall('/blog/\w+.php', rn.text)
                        news_urls = list(map(lambda x: x.split('/')[-1], news_urls))
                        news_urls = list(map(lambda x: 'https://smart-lab.ru/blog/news/' + x, news_urls))

                    except Exception as e:
                                    print(e, url)
                    for u in news_urls:
                                    global t2
                                    try:
                                

                                                res = {}
                                                nv = rq.get(u).content
                                                s = bs(nv, 'html.parser')


                                                res['time'] = s.find_all(class_="date")
                                                res['time'] = re.findall('\d+\s\w+\s\d+, \d+:\d+', str(res['time']))[0].replace(",", "")
                                                res['time'] = get_date(res['time'])

                                                res['url']= u
                                                res['site'] = 'smart-lab.ru'

                                                res['title'] = s.title.text

                                                res['text'] = s.find_all('div', class_='content')[1].text
                                                res['text'] = re.sub(re.compile('[\n\r\n]'),'', res['text'])

                                                ress.append(res)   

                           
                                    except Exception as e:
                                                        print(e, u)

        return ress

In [68]:
res2 = parse_smartlab_news()

('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/772852.php


In [71]:
df0 = pd.json_normalize(res2)
df0['text'] = df0['text'].replace(re.compile(emoji_pattern), ' ', regex=True)
df0['title'] = df0['title'].replace(re.compile(emoji_pattern), ' ', regex=True)
# df0['text'] = df0['text'].apply(lambda x: x.lower())
df0

,time,url,site,title,text
0,2023-10-23 14:40:00,https://smart-lab.ru/blog/news/952882.php,smart-lab.ru,Отказ от плавающих ставок по кредитам для бизн...,"«Большинство людей устроены так, что им хочетс..."
1,2023-10-23 12:52:00,https://smart-lab.ru/blog/news/952828.php,smart-lab.ru,Сбер увеличил срок кредита на покупку недвижим...,Сбер улучшил условия кредита на покупку недвиж...
2,2023-10-20 15:24:00,https://smart-lab.ru/blog/news/952266.php,smart-lab.ru,Дирхам вошёл в четвёрку самых популярных валют...,Купить и продать валюту ОАЭ теперь можно в 334...
3,2023-10-20 11:59:00,https://smart-lab.ru/blog/news/952169.php,smart-lab.ru,Сбербанк в 9 российских регионах предложил кли...,Сбербанк объявил о запуске в девяти регионах Р...
4,2023-10-20 11:03:00,https://smart-lab.ru/blog/news/952139.php,smart-lab.ru,"Рост потребительского кредитования, по предвар...",Рост корпоративного кредитования в РФ в сентяб...
...,...,...,...,...,...
3997,2011-03-22 13:16:00,https://smart-lab.ru/blog/news/4150.php,smart-lab.ru,Отчет Сбербанка за 2010 год,Чистая прибыль — рекордные 182 млрд рублейЭто ...
3998,2011-02-25 09:55:00,https://smart-lab.ru/blog/news/3177.php,smart-lab.ru,Сбербанк сегодня покажет класс,Два дня назад (22 февраля) я затарился Сбером ...
3999,2011-02-14 14:38:00,https://smart-lab.ru/blog/news/2664.php,smart-lab.ru,Сбербанк совместно с ЦБ в ближайшие месяцы пре...,Сбербанк совместно с Центральным банком (ЦБ) Р...
4000,2011-01-25 17:10:00,https://smart-lab.ru/blog/news/1893.php,smart-lab.ru,Сбербанк опубликовал позитивные результаты РСБ...,Сбербанк опубликовал финансовые результаты по ...


In [72]:
df0.to_csv('smartlab_SBER_new.csv', index = False)

In [74]:
def parse_smartlab_news1():
    
        ress = []
        n = rq.get('https://smart-lab.ru/forum/news/ROSN')
        news_n = re.findall('/forum/news/ROSN/page\w+', n.text)
        news_n = int(re.findall('[0-9.]+', news_n[-1])[0])

        for url in tqdm(range(1, news_n)):
                    try:   
                        global t1 

                        business_url = 'https://smart-lab.ru/forum/news/ROSN/page' + str(url)
                        rn = rq.get(business_url)
                        news_urls = re.findall('/blog/\w+.php', rn.text)
                        news_urls = list(map(lambda x: x.split('/')[-1], news_urls))
                        news_urls = list(map(lambda x: 'https://smart-lab.ru/blog/news/' + x, news_urls))

                    except Exception as e:
                                    print(e, url)
                    for u in news_urls:
                                    global t2
                                    try:
                                

                                                res = {}
                                                nv = rq.get(u).content
                                                s = bs(nv, 'html.parser')


                                                res['time'] = s.find_all(class_="date")
                                                res['time'] = re.findall('\d+\s\w+\s\d+, \d+:\d+', str(res['time']))[0].replace(",", "")
                                                res['time'] = get_date(res['time'])

                                                res['url']= u
                                                res['site'] = 'smart-lab.ru'

                                                res['title'] = s.title.text

                                                res['text'] = s.find_all('div', class_='content')[1].text
                                                res['text'] = re.sub(re.compile('[\n\r\n]'),'', res['text'])

                                                ress.append(res)   

                           
                                    except Exception as e:
                                                        print(e, u)

        return ress

In [75]:
%%time
res3 = parse_smartlab_news1()

  0%|          | 0/69 [00:00<?, ?it/s]

('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/917112.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/413941.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/239661.php
CPU times: total: 5min 26s
Wall time: 26min 2s


In [76]:
df01 = pd.json_normalize(res3)
df01['text'] = df01['text'].replace(re.compile(emoji_pattern), ' ', regex=True)
df01['title'] = df01['title'].replace(re.compile(emoji_pattern), ' ', regex=True)
# df0['text'] = df0['text'].apply(lambda x: x.lower())
df01

,time,url,site,title,text
0,2023-10-24 13:27:00,https://smart-lab.ru/blog/news/953239.php,smart-lab.ru,Россия и ОПЕК займут более 50% рынка нефти к 2...,оля России и стран ОПЕК на мировом рынке нефти...
1,2023-10-24 11:42:00,https://smart-lab.ru/blog/news/953183.php,smart-lab.ru,Бумаги Роснефти могут обновить рекорд стоимост...,Бумаги «Роснефти» могут обновить рекорд стоимо...
2,2023-10-24 11:29:00,https://smart-lab.ru/blog/news/953174.php,smart-lab.ru,Объем инвестиций в создание новой «нефтегазово...,Объем инвестиций в создание новой «нефтегазово...
3,2023-10-24 09:54:00,https://smart-lab.ru/blog/news/953121.php,smart-lab.ru,Глава МЭА предостерег энергокомпании от ставок...,С учетом «нынешней ситуации [и возможного разв...
4,2023-10-23 18:43:00,https://smart-lab.ru/blog/news/952994.php,smart-lab.ru,Евросоюз и страны G7 намерены принять юридичес...,Евросоюз и страны Группы семи (G7) намерены пр...
...,...,...,...,...,...
3442,2011-05-18 09:03:00,https://smart-lab.ru/blog/news/6837.php,smart-lab.ru,Экономические новости,ВР и «Роснефть» не прошли испытательный срок /...
3443,2011-05-17 08:30:00,https://smart-lab.ru/blog/news/6780.php,smart-lab.ru,Экономические новости,Эксперты: арест главы МВФ окажет краткосрочное...
3444,2011-05-13 07:50:00,https://smart-lab.ru/blog/news/6646.php,smart-lab.ru,Экономические новости,МГТС добавит скорости // Компания инвестирует...
3445,2011-05-10 08:29:00,https://smart-lab.ru/blog/news/6477.php,smart-lab.ru,Экономические новости,Вопрос ценыРазрушительное землетрясение в Яп...


In [77]:
df01.to_csv('smartlab_ROSN_new.csv', index = False)

In [79]:
def parse_smartlab_news2():
    
        ress = []
        n = rq.get('https://smart-lab.ru/forum/news/GAZP')
        news_n = re.findall('/forum/news/GAZP/page\w+', n.text)
        news_n = int(re.findall('[0-9.]+', news_n[-1])[0])

        for url in tqdm(range(1, news_n)):
                    try:   
                        global t1 

                        business_url = 'https://smart-lab.ru/forum/news/GAZP/page' + str(url)
                        rn = rq.get(business_url)
                        news_urls = re.findall('/blog/\w+.php', rn.text)
                        news_urls = list(map(lambda x: x.split('/')[-1], news_urls))
                        news_urls = list(map(lambda x: 'https://smart-lab.ru/blog/news/' + x, news_urls))

                    except Exception as e:
                                    print(e, url)
                    for u in news_urls:
                                    global t2
                                    try:
                                

                                                res = {}
                                                nv = rq.get(u).content
                                                s = bs(nv, 'html.parser')


                                                res['time'] = s.find_all(class_="date")
                                                res['time'] = re.findall('\d+\s\w+\s\d+, \d+:\d+', str(res['time']))[0].replace(",", "")
                                                res['time'] = get_date(res['time'])

                                                res['url']= u
                                                res['site'] = 'smart-lab.ru'

                                                res['title'] = s.title.text

                                                res['text'] = s.find_all('div', class_='content')[1].text
                                                res['text'] = re.sub(re.compile('[\n\r\n]'),'', res['text'])

                                                ress.append(res)   

                           
                                    except Exception as e:
                                                        print(e, u)

        return ress

In [80]:
%%time
res4 = parse_smartlab_news2()

  0%|          | 0/146 [00:00<?, ?it/s]

('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/888316.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/875138.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/874226.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/837838.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/759197.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существую

In [81]:
df02 = pd.json_normalize(res4)
df02['text'] = df02['text'].replace(re.compile(emoji_pattern), ' ', regex=True)
df02['title'] = df02['title'].replace(re.compile(emoji_pattern), ' ', regex=True)
# df0['text'] = df0['text'].apply(lambda x: x.lower())
df02

,time,url,site,title,text
0,2023-10-24 16:49:00,https://smart-lab.ru/blog/news/953317.php,smart-lab.ru,"Газпром Капитал, ЗО24-1-Е - возможность для вх...",Самый активный эмитент замещающих облигаций — ...
1,2023-10-24 13:31:00,https://smart-lab.ru/blog/news/953243.php,smart-lab.ru,Госдума приняла закон об экспорте СПГ без прив...,В начале июня с таким предложением выступил гл...
2,2023-10-24 12:36:00,https://smart-lab.ru/blog/news/953218.php,smart-lab.ru,"Строительство газопровода ""Союз Восток"" через ...",Строительство газопровода «Союз Восток» через ...
3,2023-10-24 12:35:00,https://smart-lab.ru/blog/news/953217.php,smart-lab.ru,Газпром готов принять участие в газификации Мо...,«Газпром» готов принять участие в газификации ...
4,2023-10-24 11:46:00,https://smart-lab.ru/blog/news/953187.php,smart-lab.ru,Новые СПГ-проекты ограничат возможности РФ в п...,"Новые мировые проекты по сжижению газа, которы..."
...,...,...,...,...,...
7272,2011-07-11 09:24:00,https://smart-lab.ru/blog/news/10128.php,smart-lab.ru,Обзор утренних новостей,«Мечел»Аналитики Bank of America Merrill Lynch...
7273,2011-07-08 08:22:00,https://smart-lab.ru/blog/news/10021.php,smart-lab.ru,Обзор новостей на утро,НЛМКRaiffeisen повысили рекомендацию по распис...
7274,2011-07-07 07:43:00,https://smart-lab.ru/blog/news/9931.php,smart-lab.ru,Новости экономики на утро,Сбербанк столкнулся с Австро-Венгрией // На Vo...
7275,2011-07-05 09:08:00,https://smart-lab.ru/blog/news/9808.php,smart-lab.ru,Обзор экономичеких новостей на утро,«Аэрофлот»Deutsche Bank повысил прогнозную цен...


In [82]:
df02.to_csv('smartlab_GAZP_new.csv', index = False)

In [84]:
def parse_smartlab_news3():
    
        ress = []
        n = rq.get('https://smart-lab.ru/forum/news/MOEX')
        news_n = re.findall('/forum/news/MOEX/page\w+', n.text)
        news_n = int(re.findall('[0-9.]+', news_n[-1])[0])

        for url in tqdm(range(1, news_n)):
                    try:   
                        global t1 

                        business_url = 'https://smart-lab.ru/forum/news/MOEX/page' + str(url)
                        rn = rq.get(business_url)
                        news_urls = re.findall('/blog/\w+.php', rn.text)
                        news_urls = list(map(lambda x: x.split('/')[-1], news_urls))
                        news_urls = list(map(lambda x: 'https://smart-lab.ru/blog/news/' + x, news_urls))

                    except Exception as e:
                                    print(e, url)
                    for u in news_urls:
                                    global t2
                                    try:
                                

                                                res = {}
                                                nv = rq.get(u).content
                                                s = bs(nv, 'html.parser')


                                                res['time'] = s.find_all(class_="date")
                                                res['time'] = re.findall('\d+\s\w+\s\d+, \d+:\d+', str(res['time']))[0].replace(",", "")
                                                res['time'] = get_date(res['time'])

                                                res['url']= u
                                                res['site'] = 'smart-lab.ru'

                                                res['title'] = s.title.text

                                                res['text'] = s.find_all('div', class_='content')[1].text
                                                res['text'] = re.sub(re.compile('[\n\r\n]'),'', res['text'])

                                                ress.append(res)   

                           
                                    except Exception as e:
                                                        print(e, u)

        return ress

In [85]:
%%time
res5 = parse_smartlab_news3()

  0%|          | 0/41 [00:00<?, ?it/s]

list index out of range https://smart-lab.ru/blog/news/718348.php
list index out of range https://smart-lab.ru/blog/news/706275.php
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)) https://smart-lab.ru/blog/news/630862.php
CPU times: total: 3min 23s
Wall time: 16min 1s


In [86]:
df03 = pd.json_normalize(res5)
df03['text'] = df03['text'].replace(re.compile(emoji_pattern), ' ', regex=True)
df03['title'] = df03['title'].replace(re.compile(emoji_pattern), ' ', regex=True)
# df0['text'] = df0['text'].apply(lambda x: x.lower())
df03

,time,url,site,title,text
0,2023-10-24 15:16:00,https://smart-lab.ru/blog/news/953281.php,smart-lab.ru,"""Детский мир"" напоминает, что 3 ноября 2023г з...","ГК «Детский мир» (далее – «Компания», MOEX: DS..."
1,2023-10-24 14:23:00,https://smart-lab.ru/blog/news/953259.php,smart-lab.ru,Власти одобрили выкуп компанией Cофтлайн доли ...,Капитализация: RUB 48 966 млн В свободном обра...
2,2023-10-23 18:05:00,https://smart-lab.ru/blog/news/952975.php,smart-lab.ru,23 октября 2023г Национальная товарная биржа (...,23 октября 2023 года Национальная товарная бир...
3,2023-10-23 17:32:00,https://smart-lab.ru/blog/news/952960.php,smart-lab.ru,7 ноября 2023г вступает в силу новая редакция ...,7 ноября 2023 года вступает в силу новая редак...
4,2023-10-23 15:34:00,https://smart-lab.ru/blog/news/952908.php,smart-lab.ru,После делистинга с Nasdaq бумаги Ozon продолжа...,Ozon – Компания объявила о добровольном делист...
...,...,...,...,...,...
2042,2016-04-18 16:22:00,https://smart-lab.ru/blog/news/323017.php,smart-lab.ru,Ежеквартальный пересмотр шага цены на Московск...,С целью повышения эффективности микроструктуры...
2043,2016-04-04 21:02:00,https://smart-lab.ru/blog/news/320270.php,smart-lab.ru,Объемы торгов резко выросли на всех рынка Моск...,"Общий оборот: +43,4% г/г = 74,7 трлн рублСрочн..."
2044,2016-03-21 13:30:00,https://smart-lab.ru/blog/news/317594.php,smart-lab.ru,Сайт московской биржи недоступен,Сайт Московской биржи недоступен из-за техниче...
2045,2016-02-05 12:28:00,https://smart-lab.ru/blog/news/308696.php,smart-lab.ru,Московская биржа начинает публикацию индексов ...,10 февраля 2016 года Московская биржа начинает...


In [87]:
df03.to_csv('smartlab_MOEX_new.csv', index = False)

In [5]:
def parse_smartlab_news4():
    
        ress = []
        n = rq.get('https://smart-lab.ru/forum/news/RTKM')
        news_n = re.findall('/forum/news/RTKM/page\w+', n.text)
        news_n = int(re.findall('[0-9.]+', news_n[-1])[0])

        for url in tqdm(range(1, news_n)):
                    try:   
                        global t1 

                        business_url = 'https://smart-lab.ru/forum/news/RTKM/page' + str(url)
                        rn = rq.get(business_url)
                        news_urls = re.findall('/blog/\w+.php', rn.text)
                        news_urls = list(map(lambda x: x.split('/')[-1], news_urls))
                        news_urls = list(map(lambda x: 'https://smart-lab.ru/blog/news/' + x, news_urls))

                    except Exception as e:
                                    print(e, url)
                    for u in news_urls:
                                    global t2
                                    try:
                                

                                                res = {}
                                                nv = rq.get(u).content
                                                s = bs(nv, 'html.parser')


                                                res['time'] = s.find_all(class_="date")
                                                res['time'] = re.findall('\d+\s\w+\s\d+, \d+:\d+', str(res['time']))[0].replace(",", "")
                                                res['time'] = get_date(res['time'])

                                                res['url']= u
                                                res['site'] = 'smart-lab.ru'

                                                res['title'] = s.title.text

                                                res['text'] = s.find_all('div', class_='content')[1].text
                                                res['text'] = re.sub(re.compile('[\n\r\n]'),'', res['text'])

                                                ress.append(res)   

                           
                                    except Exception as e:
                                                        print(e, u)

        return ress

In [6]:
%%time
res6 = parse_smartlab_news4()

  0%|          | 0/18 [00:00<?, ?it/s]

CPU times: total: 10min 36s
Wall time: 16min 22s


In [7]:
df04 = pd.json_normalize(res6)
df04['text'] = df04['text'].replace(re.compile(emoji_pattern), ' ', regex=True)
df04['title'] = df04['title'].replace(re.compile(emoji_pattern), ' ', regex=True)
# df0['text'] = df0['text'].apply(lambda x: x.lower())
df04

,time,url,site,title,text
0,2023-11-09 14:46:00,https://smart-lab.ru/blog/news/958445.php,smart-lab.ru,Ростелеком сейчас интересен для добавления в п...,ПАО «Ростелеком» объявляет результаты деятельн...
1,2023-11-09 09:59:00,https://smart-lab.ru/blog/news/958290.php,smart-lab.ru,Выручка Ростелекома за 9 месяцев по МСФО вырос...,Выручка российского провайдера цифровых услуг ...
2,2023-11-08 11:37:00,https://smart-lab.ru/blog/news/957897.php,smart-lab.ru,Шведский правообладатель разрешил российскому ...,Шведская Tele2 Sverige AB разрешила российском...
3,2023-11-07 18:26:00,https://smart-lab.ru/blog/news/957705.php,smart-lab.ru,Таблица по дивидендам от РБК Инвестиций,t.me/selfinvestor
4,2023-11-07 14:21:00,https://smart-lab.ru/blog/news/957606.php,smart-lab.ru,Стратегия развития отрасли связи до 2035г долж...,Стратегия развития отрасли связи до 2035г долж...
...,...,...,...,...,...
894,2011-05-11 08:35:00,https://smart-lab.ru/blog/news/6531.php,smart-lab.ru,Экономические новости,Евро держится на американском слове // Греция...
895,2011-05-10 17:35:00,https://smart-lab.ru/blog/news/6503.php,smart-lab.ru,"Объединенный Ростелеком - новая ""фишка""...",По информации от моих коллег (Связьинвест) в б...
896,2011-04-28 07:57:00,https://smart-lab.ru/blog/news/6033.php,smart-lab.ru,Экономические новости,«Русгидро» запутала рынок // учетом выручки ...
897,2011-04-26 21:06:00,https://smart-lab.ru/blog/news/5951.php,smart-lab.ru,Юрченко планирует увеличить долю в Ростелекоме...,Бывший глава Связьинвеста Евгений Юрченко план...


In [8]:
df04.to_csv('smartlab_RTKM_new.csv', index = False)